In [1]:
import os

import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

from tqdm.notebook import tqdm
import numpy as np
import pdb

from langchain.document_loaders import DirectoryLoader

os.chdir('..\\') 
from testGPT.model import GPTConfig, GPT
from testGPT.train import get_batch, estimate_loss
from testGPT.data.char_level import CharEncDec

In [2]:
DATA_PATH = "../data/text/"

def load_documents():
    loader = DirectoryLoader(DATA_PATH, glob="*.pdf")
    documents = loader.load()
    return documents

docs = load_documents()

In [3]:
"Camões"
chunks = [x.page_content.split('Luís de Camões')[-1] if 'Glosa' not in x.page_content else x.page_content.split('38. Glosa')[-1] for x in docs ]
text = " ".join(chunks)

In [4]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  572017


In [5]:
print(text[:1000])



1. Canção

Fermosa e gentil Dama, quando vejo a testa de ouro e neve, o lindo aspeito, a boca graciosa, o riso honesto, o colo de cristal, o branco peito, de meu não quero mais que meu desejo, nem mais de vós que ver tão lindo gesto. Ali me manifesto por vosso a Deus e ao mundo; ali me inflamo nas lágrimas que choro, e de mim, que vos amo, em ver que soube amar-vos, me namoro; e fico por mim só perdido, de arte que hei ciúmes de mim por vossa parte.

Se porventura vivo descontente por fraqueza d'esprito, padecendo a doce pena que entender não sei, fujo de mim e acolho-me, correndo, à vossa vista; e fico tão contente que zombo dos tormentos que passei. De quem me queixarei se vós me dais a vida deste jeito nos males que padeço, senão de meu sujeito,

que não cabe com bem de tanto preço? Mas inda isso de mim cuidar não posso, de estar muito soberbo com ser vosso.

Se, por algum acerto, Amor vos erra por parte do desejo, cometendo algum nefando e torpe desatino, se ainda mais que ver, e

In [6]:
tokenizer = CharEncDec(text)

In [7]:
print(''.join(tokenizer.chars))
print(tokenizer.vocab_size)


 !"&'()*,-./0123456789:;<>?ABCDEFGHIJLMNOPQRSTUVWXYZ[]`abcdefghijlmnopqrstuvwxyz{|}~¡ª«»¿ÀÁÃÇÉÍÓÔÕÜàáâãçèéêëìíïñòóôõùúü–—’
123


In [10]:
tokenizer.enconde

<bound method CharEncDec.enconde of <testGPT.data.char_level.CharEncDec object at 0x000001FA183D9B90>>

In [8]:
# create a mapping from characters to integers

print(tokenizer.encode(text[:500]))
print(tokenizer.decode(encode(text[:500])))

AttributeError: 'CharEncDec' object has no attribute 'encode'

In [9]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:500])

torch.Size([572017]) torch.int64
tensor([  0,   0,  14,  11,   1,  30,  56,  68, 104, 103,  69,   0,   0,  33,
         60,  72,  67,  69,  73,  56,   1,  60,   1,  62,  60,  68,  74,  64,
         66,   1,  31,  56,  67,  56,   9,   1,  71,  75,  56,  68,  59,  69,
          1,  76,  60,  65,  69,   1,  56,   1,  74,  60,  73,  74,  56,   1,
         59,  60,   1,  69,  75,  72,  69,   1,  60,   1,  68,  60,  76,  60,
          9,   1,  69,   1,  66,  64,  68,  59,  69,   1,  56,  73,  70,  60,
         64,  74,  69,   9,   1,  56,   1,  57,  69,  58,  56,   1,  62,  72,
         56,  58,  64,  69,  73,  56,   9,   1,  69,   1,  72,  64,  73,  69,
          1,  63,  69,  68,  60,  73,  74,  69,   9,   1,  69,   1,  58,  69,
         66,  69,   1,  59,  60,   1,  58,  72,  64,  73,  74,  56,  66,   9,
          1,  69,   1,  57,  72,  56,  68,  58,  69,   1,  70,  60,  64,  74,
         69,   9,   1,  59,  60,   1,  67,  60,  75,   1,  68, 103,  69,   1,
         71,  75,  60,  72,  69

In [10]:
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [11]:
config = GPTConfig()
block_size = 128
config.block_size = block_size
config.vocab_size = vocab_size
config.n_layer = 3
config.n_head = 4
config.n_embd = 32
config.dropout = 0.1


batch_size = 32 # how many independent sequences will we process in parallel?
max_iters = 200000
eval_interval = 500
learning_rate = 1e-3
eval_iters = 200
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = GPT(config)
m = model.to(device)
xb, yb = get_batch(train_data, batch_size, block_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long, device=device), max_new_tokens=100)[0].tolist()))
print('Num params: ', m.get_num_params())

torch.Size([4096, 123])
tensor(4.9798, grad_fn=<NllLossBackward0>)

á/ostu<U»0|ÜIa' tÉX/wúá~u*GI'3—ô—~G>31Üh»ad{1â32DÍÍ}ÜUtì7v;X&ÃÁ|9ÜMRIX)Ç`fWíFzã63çYwïò`Édfl~6:à«’aw7
Num params:  45883


In [12]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)
for iter in tqdm(range(max_iters)):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss(m, eval_iters, train_data, val_data, batch_size, block_size)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch(train_data, batch_size, block_size)

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

  0%|          | 0/200000 [00:00<?, ?it/s]

step 0: train loss 4.9757, val loss 4.9683
step 500: train loss 2.3887, val loss 2.3607



KeyboardInterrupt



In [48]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


«Damosos

a pralha o que se vejo vós almaros no as dita da gente sebo vas enclta íreros varoveer neliá da Lusitando, Deptundo, Que assi rompanha Elegas a Nunos altos senão ter muitar namado; e, que os tabalha, juyeita estrasse suavar tuemente Desos vossoeles que que quero manha come tonsidadas; E, buscandas, comeno endo que lhamada extrem corra, enteja que crievo quem tinha por poder a Nocerto, e nas como apio dor parço a farra estaráro; se deiente escuro, Inúmonho Senhora no suada, Faras nas, c
